In [24]:
import os

In [25]:
%pwd

'c:\\Projects\\Kidney_Disease_Classification'

In [3]:
os.chdir('../')

In [ ]:
%pwd

'c:\\Projects\\Kidney_Disease_Classification'

In [27]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size:list



In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [30]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs= dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode="nearest",
                **datagenerator_kwargs
            )

        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        print(next(iter(self.train_generator))[0].shape)

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)



    def train(self):
        self.get_base_model()   # load model before training

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            epochs=self.config.params_epochs
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

      

       

In [32]:
try:
    training_config = ConfigurationManager().get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    

except Exception as e:
    raise e

[2025-05-31 10:51:18,949 : INFO  :common : yaml file: config\config.yaml loaded successfully]
[2025-05-31 10:51:18,958 : INFO  :common : yaml file: params.yaml loaded successfully]
[2025-05-31 10:51:18,963 : INFO  :common : created directory at: artifacts]
[2025-05-31 10:51:18,969 : INFO  :common : created directory at: artifacts\training]


Found 2483 images belonging to 4 classes.
Found 9940 images belonging to 4 classes.
(16, 224, 224, 3)
621/621 [==============================] - 2092s 3s/step - loss: 13.8672 - accuracy: 0.4169 - val_loss: 23.7751 - val_accuracy: 0.4367
